In [1]:
pip install deepdiff

  Using cached ordered_set-4.1.0-py3-none-any.whl (7.6 kB)
Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install ppdeep

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install ssdeep

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 813.0 kB/s eta 0:00:00 MB/s eta 0:00:01
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [128 lines of output]
      /home/harsh/miniconda3/lib/python3.11/site-packages/setuptools/__init__.py:84: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
      !!
      
              ********************************************************************************
              Requirements should be satisfied by a PEP 517 installer.
              If you are using pip, you can try `pip install --use-pep517`.
              ********************************************************************************
      
      !!
        dist.fetch_build_eggs(dist.setup_requires)
      running egg_info
      creating /tmp/pip-pip-egg-info-seunzzw4/ssdeep.egg-info
      writing /tmp/pip-pip-egg-info-seunzzw4/ssde

In [113]:
import os
from io import BytesIO


BLOCKSIZE_MIN = 3
SPAMSUM_LENGTH = 64


def _spamsum(stream, slen):
	STREAM_BUFF_SIZE = 8192
	HASH_PRIME = 0x01000193
	HASH_INIT = 0x28021967
	ROLL_WINDOW = 7
	B64 = 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+/'

	roll_win = bytearray(ROLL_WINDOW)
	roll_h1 = int()
	roll_h2 = int()
	roll_h3 = int()
	roll_n = int()
	block_size = int()
	hash_string1 = str()
	hash_string2 = str()
	block_hash1 = int(HASH_INIT)
	block_hash2 = int(HASH_INIT)

	bs = BLOCKSIZE_MIN
	while (bs * SPAMSUM_LENGTH) < slen:
		bs = bs * 2
	block_size = bs

	while True:
		if block_size < BLOCKSIZE_MIN:
			raise RuntimeError('Calculated block size is too small')

		stream.seek(0)
		buf = stream.read(STREAM_BUFF_SIZE)
		while buf:
			for b in buf:
				block_hash1 = ((block_hash1 * HASH_PRIME) & 0xFFFFFFFF) ^ b
				block_hash2 = ((block_hash2 * HASH_PRIME) & 0xFFFFFFFF) ^ b

				roll_h2 = roll_h2 - roll_h1 + (ROLL_WINDOW * b)
				roll_h1 = roll_h1 + b - roll_win[roll_n % ROLL_WINDOW]
				roll_win[roll_n % ROLL_WINDOW] = b
				roll_n += 1
				roll_h3 = (roll_h3 << 5) & 0xFFFFFFFF
				roll_h3 ^= b

				rh = roll_h1 + roll_h2 + roll_h3

				if (rh % block_size) == (block_size - 1):
					if len(hash_string1) < (SPAMSUM_LENGTH - 1):
						hash_string1 += B64[block_hash1 % 64]
						block_hash1 = HASH_INIT
					if (rh % (block_size * 2)) == ((block_size * 2) - 1):
						if len(hash_string2) < ((SPAMSUM_LENGTH // 2) - 1):
							hash_string2 += B64[block_hash2 % 64]
							block_hash2 = HASH_INIT

			buf = stream.read(STREAM_BUFF_SIZE)

		if block_size > BLOCKSIZE_MIN and len(hash_string1) < (SPAMSUM_LENGTH // 2):
			block_size = (block_size // 2)

			roll_win = bytearray(ROLL_WINDOW)
			roll_h1 = roll_h2 = roll_h3 = int()
			roll_n = int()

			block_hash1 = block_hash2 = HASH_INIT
			hash_string1 = hash_string2 = str()
		else:
			rh = roll_h1 + roll_h2 + roll_h3
			if rh != 0:
				hash_string1 += B64[block_hash1 % 64]
				hash_string2 += B64[block_hash2 % 64]
			break

	return '{0}:{1}:{2}'.format(block_size, hash_string1, hash_string2)


def hash(buf):
	if isinstance(buf, bytes):
		pass
	elif isinstance(buf, str):
		buf = buf.encode()
	else:
		raise TypeError('Argument must be of bytes or string type, not %r' % type(buf))
	return _spamsum(BytesIO(buf), len(buf))


def hash_from_file(filename):
	if not isinstance(filename, str):
		raise TypeError('Argument must be of string type, not %r' % type(filename))
	if not os.path.isfile(filename):
		raise IOError('File not found')
	if not os.access(filename, os.R_OK):
		raise IOError('File is not readable')
	fsize = os.stat(filename).st_size
	return _spamsum(open(filename, 'rb'), fsize)


def _levenshtein(s, t):
	'''
	Implementation by Christopher P. Matthews
	'''
	if s == t: return 0
	elif len(s) == 0: return len(t)
	elif len(t) == 0: return len(s)
	v0 = [None] * (len(t) + 1)
	v1 = [None] * (len(t) + 1)
	for i in range(len(v0)):
		v0[i] = i
	for i in range(len(s)):
		v1[0] = i + 1
		for j in range(len(t)):
			cost = 0 if s[i] == t[j] else 1
			v1[j + 1] = min(v1[j] + 1, v0[j + 1] + 1, v0[j] + cost)
		for j in range(len(v0)):
			v0[j] = v1[j]
	return v1[len(t)]


class _RollState(object):
	ROLL_WINDOW = 7

	def __init__(self):
		self.win = bytearray(self.ROLL_WINDOW)
		self.h1 = int()
		self.h2 = int()
		self.h3 = int()
		self.n = int()

	def roll_hash(self, b):
		self.h2 = self.h2 - self.h1 + (self.ROLL_WINDOW * b)
		self.h1 = self.h1 + b - self.win[self.n % self.ROLL_WINDOW]
		self.win[self.n % self.ROLL_WINDOW] = b
		self.n += 1
		self.h3 = (self.h3 << 5) & 0xFFFFFFFF
		self.h3 ^= b
		return self.h1 + self.h2 + self.h3


def _common_substring(s1, s2):
	ROLL_WINDOW = 7
	hashes = list()

	roll = _RollState()
	for i in range(len(s1)):
		b = ord(s1[i])
		hashes.append(roll.roll_hash(b))

	roll = _RollState()
	for i in range(len(s2)):
		b = ord(s2[i])
		rh = roll.roll_hash(b)
		if i < (ROLL_WINDOW - 1):
			continue
		for j in range(ROLL_WINDOW-1, len(hashes)):
			if hashes[j] != 0 and hashes[j] == rh:
				ir = i - (ROLL_WINDOW - 1)
				jr = j - (ROLL_WINDOW - 1)
				if (len(s2[ir:]) >= ROLL_WINDOW and
					s2[ir:ir+ROLL_WINDOW] == s1[jr:jr+ROLL_WINDOW]):
					return True
	return False


def _score_strings(s1, s2, block_size):
	if _common_substring(s1, s2) == False:
		return 0
	score = _levenshtein(s1, s2)
	score = (score * SPAMSUM_LENGTH) // (len(s1) + len(s2))
	score = (100 * score) // SPAMSUM_LENGTH
	score = 100 - score
	if score > (block_size // BLOCKSIZE_MIN * min([len(s1), len(s2)])):
		score = block_size // BLOCKSIZE_MIN * min([len(s1), len(s2)])
	return score


def _strip_sequences(s):
	r = s[:3]
	for i in range(3, len(s)):
		if (s[i] != s[i-1] or s[i] != s[i-2] or s[i] != s[i-3]):
			r += s[i]
	return r


def compare(hash1, hash2):
	if not (isinstance(hash1, str) and isinstance(hash2, str)):
		raise TypeError('Arguments must be of string type')
	try:
		hash1_bs, hash1_s1, hash1_s2 = hash1.split(':')
		hash2_bs, hash2_s1, hash2_s2 = hash2.split(':')
		hash1_bs = int(hash1_bs)
		hash2_bs = int(hash2_bs)
	except ValueError:
		raise ValueError('Invalid hash format') from None

	if hash1_bs != hash2_bs and hash1_bs != (hash2_bs * 2) and hash2_bs != (hash1_bs * 2):
		return 0

	hash1_s1 = _strip_sequences(hash1_s1)
	hash1_s2 = _strip_sequences(hash1_s2)
	hash2_s1 = _strip_sequences(hash2_s1)
	hash2_s2 = _strip_sequences(hash2_s2)

	if hash1_bs == hash2_bs and hash1_s1 == hash2_s1:
		return 100

	if hash1_bs == hash2_bs:
		score1 = _score_strings(hash1_s1, hash2_s1, hash1_bs)
		score2 = _score_strings(hash1_s2, hash2_s2, hash2_bs)
		score = int(max([score1, score2]))
		return score
	elif hash1_bs == (hash2_bs * 2):
		score = int(_score_strings(hash1_s1, hash2_s2, hash1_bs))
		return score
	else:
		score = int(_score_strings(hash1_s2, hash2_s1, hash2_bs))
		return score
	return 0

In [2]:
import os
import json
import ssdeep

In [53]:
folders = os.listdir(folder_path)

['Abbottabad_Falcons',
 'ACC_Asia_XI_cricket_team',
 'Adelaide_Strikers',
 'Adelaide_Strikers_(WBBL)',
 'Afghanistan_A_cricket_team',
 'Afghanistan_national_cricket_team',
 'Afghanistan_national_under-19_cricket_team',
 'Afghan_Cheetahs',
 'Africa_XI_cricket_team',
 'Ahmedabad_Rockets',
 'AJK_Jaguars',
 'Antigua_and_Barbuda_national_cricket_team',
 'Antigua_Hawksbills',
 'Argentina_national_cricket_team',
 'Arunachal_Pradesh_cricket_team',
 'Australia_national_cricket_team',
 'Austria_national_cricket_team',
 'Badureliya_Sports_Club',
 'Bagh_Stallions',
 'Bahamas_national_cricket_team',
 'Bahawalpur_Stags',
 'Bahrain_national_cricket_team',
 'Balkh_Legends',
 'Balochistan_cricket_team',
 'Band-e-Amir_Dragons',
 'Bangladesh_national_cricket_team',
 'Bangladesh_national_under-19_cricket_team',
 'Bankstown_District_Cricket_Club',
 'Barbados_Royals',
 'Barbados_Royals_(WCPL)',
 'Basnahira_Cricket_Dundee',
 'Basnahira_North_cricket_team',
 'Basnahira_South_cricket_team',
 'Bath_Cricket_Club

In [27]:
files = os.listdir(folder_path)
files_int = []
for filename in files:
    if filename!="logs.txt":
        filen = int(filename.split(".")[0])
        files_int.append(filen)
files_int.sort()

In [28]:
files_int

[587,
 709,
 719,
 747,
 752,
 773,
 860,
 1010,
 1018,
 1020,
 1021,
 1022,
 1052,
 1075,
 1168,
 1171,
 1178,
 1279,
 1280,
 1286,
 1288,
 1315,
 1324,
 1325,
 1344,
 1345,
 1346,
 1353,
 1363,
 1364,
 1366,
 1367,
 1368,
 1369,
 1370,
 1371,
 1372,
 1379,
 1380,
 1427,
 1475,
 1480,
 1481,
 1483,
 1512,
 1513,
 1514,
 1534,
 1535,
 1536,
 1537,
 1556,
 1560,
 1561,
 1566,
 1567,
 1610,
 1613,
 1614,
 1615,
 1616,
 1628,
 1629,
 1630,
 1642,
 1654,
 1655,
 1664,
 1666,
 1667,
 1671,
 1672,
 1673,
 1674,
 1675,
 1676,
 1677,
 1685,
 1690,
 1691,
 1692,
 1705,
 1743,
 1745,
 1758,
 1765,
 1766,
 1767,
 1774,
 1775,
 1776,
 1777,
 1781,
 1797,
 1846,
 1850,
 1851,
 1853,
 1855,
 1857,
 1888,
 1889,
 1927,
 1929,
 1959,
 1977,
 1993,
 2020,
 2040,
 2042,
 2057,
 2107,
 2108,
 2115,
 2136,
 2166,
 2169,
 2170,
 2171,
 2192,
 2197,
 2202,
 2203,
 2204,
 2214,
 2216,
 2219,
 2220,
 2221,
 2222,
 2256,
 2310,
 2312,
 2328,
 2329,
 2384,
 2387,
 2392,
 2393,
 2394,
 2399,
 2400,
 2401,
 2425,

In [31]:
pip install jellyfish

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 5.2 MB/s eta 0:00:00m eta 0:00:010:01:01
Note: you may need to restart the kernel to use updated packages.


In [32]:
import jellyfish

In [34]:
jellyfish.levenshtein_distance("Jane Halton","Rosemary Huxtable")

13

In [35]:
jellyfish.jaro_similarity("Jane Halton","Rosemary Huxtable")

0.5216874628639334

In [40]:
jellyfish.jaro_winkler_similarity("Justin Langer","Justin Langerhgfhgdfvxxjvfghxzfghhxh")

0.8722222222222222

In [30]:
h1 = ssdeep.hash("Jane Halton")
h2 = ssdeep.hash("Rosemary Huxtable")
fuzzy = ssdeep.compare(h1,h2)
fuzzy

0

In [86]:
#cricket team

important_keys = ['name','captain','test_captain','od_captain','t20i_captain','1captain','2captain','coach','test_coach',
                  'od_coach','chairman','batting_coach','bowling_coach','fielding_coach','overseas','overseas1','overseas2',
                  'owner','ceo','manager','adviser','num_titles','title1','title1wins','title2','title2wins','title3','title3wins',
                  'title4','title4wins','title5','title5wins','title6','title6wins','title7','title7wins','title8','title8wins',
                  'sheffield','frc_wins','t20_wins','ipl_wins','clt20_wins','bpl_wins','psl_wins','WCL_division','test_rank',
                  'test_rank_best','odi_rank','odi_rank_best','t20i_rank','t20i_rank_best','wodi_rank','wodi_rank_best','wt20i_rank',
                  'wt20i_rank_best','num_tests','test_record','num_tests_this_year','test_record_this_year','wtc_apps','num_odis',
                  'odi_record','num_odis_this_year','odi_record_this_year','wc_apps','num_t20is','t20i_record','num_t20is_this_year',
                  't20i_record_this_year','wt20_apps','wcq_apps','wcq_best','wt20q_apps','wt20q_best','num_wtests','wtest_record',
                  'num_wtests_this_year','wtest_record_this_year','num_wodis','wodi_record','num_wodis_this_year','wodi_record_this_year',
                  'wwc_apps','wwc_best','num_wt20is','num_wt20is_this_year','wt20i_record_this_year','wwt20_apps','wwt20_best']

bounded_similarity = ['name','captain','test_captain','od_captain','t20i_captain','1captain','2captain','coach','test_coach',
                  'od_coach','chairman','batting_coach','bowling_coach','fielding_coach','overseas','overseas1','overseas2',
                  'owner','ceo','manager','adviser']


In [85]:
# gov agency

important_keys = ['name','employees','budget','minister_type','minister1_name','minister1_pfo','minister2_name','minister2_pfo','minister3_name','minister3_pfo','minister4_name','minister4_pfo','minister5_name','minister5_pfo','minister6_name','minister6_pfo','minister7_name','minister7_pfo','minister8_name','minister8_pfo','deputyminister_type','deputyminister1_name','deputyminister1_pfo','deputyminister2_name','deputyminister2_pfo','deputyminister3_name','deputyminister3_pfo','deputyminister4_name','deputyminister4_pfo','deputyminister5_name','deputyminister5_pfo','deputyminister6_name','deputyminister6_pfo','deputyminister7_name','deputyminister7_pfo','deputyminister8_name','deputyminister8_pfo','chief1_name','chief1_position','chief2_name','chief2_position','chief3_name','chief3_position','chief4_name','chief4_position','chief5_name','chief5_position','chief6_name','chief6_position','chief7_name','chief7_position','chief8_name','chief8_position','chief9_name','chief9_position']
bounded_similarity = ['name','minister_type','minister1_name','minister1_pfo','minister2_name','minister2_pfo','minister3_name','minister3_pfo','minister4_name','minister4_pfo','minister5_name','minister5_pfo','minister6_name','minister6_pfo','minister7_name','minister7_pfo','minister8_name','minister8_pfo','deputyminister_type','deputyminister1_name','deputyminister1_pfo','deputyminister2_name','deputyminister2_pfo','deputyminister3_name','deputyminister3_pfo','deputyminister4_name','deputyminister4_pfo','deputyminister5_name','deputyminister5_pfo','deputyminister6_name','deputyminister6_pfo','deputyminister7_name','deputyminister7_pfo','deputyminister8_name','deputyminister8_pfo','chief1_name','chief1_position','chief2_name','chief2_position','chief3_name','chief3_position','chief4_name','chief4_position','chief5_name','chief5_position','chief6_name','chief6_position','chief7_name','chief7_position','chief8_name','chief8_position','chief9_name','chief9_position'] 

In [83]:
##university

important_keys = ['name','endowment','budget','officer_in_charge','chair','chairman','chairperson','visitor','chancellor',
                  'president','vice_president','superintendent','vice_chancellor','provost','rector','principal','director','dean',
                  'head_label','head','academic_staff','total_staff','students','undergrad','postgrad','doctoral','other_students','other']
bounded_similarity = ['name','officer_in_charge','chair','chairman','chairperson','visitor','chancellor',
                  'president','vice_president','superintendent','vice_chancellor','provost','rector','principal','director','dean',
                  'head_label','head']

In [87]:
## country

important_keys = ['common_name','largest_city','leader_title1','leader_name1','leader_title2','leader_name2','leader_title3',
                  'leader_name3','leader_title4','leader_name4','leader_title5','leader_name5','leader_title6','leader_name6',
                  'leader_title7','leader_name7','leader_title8','leader_name8','leader_title9','leader_name9','leader_title10',
                  'leader_name10','leader_title11','leader_name11','leader_title12','leader_name12','leader_title13','leader_name13',
                  'leader_title14','leader_name14','population_estimate','population_estimate_rank','population_estimate_year',
                  'population_census','population_census_year','population_density_km2','population_density_sq_mi',
                  'population_density_rank','GDP_PPP','GDP_PPP_rank','GDP_PPP_year','GDP_PPP_per_capita','GDP_PPP_per_capita_rank',
                  'GDP_nominal','GDP_nominal_rank','GDP_nominal_year','GDP_nominal_per_capita','GDP_nominal_per_capita_rank','Gini',
                  'Gini_rank','Gini_year','HDI_year','HDI','HDI_rank']

bounded_similarity = ['common_name','largest_city','leader_title1','leader_name1','leader_title2','leader_name2','leader_title3',
                  'leader_name3','leader_title4','leader_name4','leader_title5','leader_name5','leader_title6','leader_name6',
                  'leader_title7','leader_name7','leader_title8','leader_name8','leader_title9','leader_name9','leader_title10',
                  'leader_name10','leader_title11','leader_name11','leader_title12','leader_name12','leader_title13','leader_name13',
                  'leader_title14','leader_name14',]

In [79]:
folders_path = "info_country/infoboxes/new_countries"

In [80]:
number_of_changing_keys = 3
# fuzzy_hash_match = 80  ##100 full match
jaro_lower_bound = 0.35
jaro_upper_bound = 0.85
minimum_number_of_tables = 5

In [81]:
folders = os.listdir(folders_path)
bad_data = 0
less_than_two_file = []
changing_keys = set()
for folder in folders:
    if folder == "json_compare_logging.txt" or folder == "good_folders.txt":
        continue
    print(folder)
    folder_path = os.path.join(folders_path,folder)
    files = os.listdir(folder_path)
    files_int = []
    for filename in files:
        if filename!="logs.txt" and filename!=f"{folder}.json":
            filen = int(filename.split(".")[0])
            files_int.append(filen)
    files_int.sort()
        
    if len(files_int)<2:
        less_than_two_file.append(folder)
        continue
    final_json = {}
    first = 0
    second = 1
    
    while second < len(files_int):
        with open(f"{folder_path}/{files_int[first]}.json",'r',encoding = "utf-8") as f:
            try:
                data_first = json.load(f)
            except:
                print(first)
        with open(f"{folder_path}/{files_int[second]}.json",'r',encoding = "utf-8") as f:
            try:
                data_second = json.load(f)
            except:
                print(second)
        first_keys = data_first.keys()
        second_keys = data_second.keys()
        number_of_keys = min(len(first_keys),len(second_keys))
        count = 0
        count1 = 0
        js = {}
        for key in second_keys:
            if key in first_keys and key in important_keys:
                if not isinstance(data_first[key],list) and not isinstance(data_second[key],list):
                    similarity = jellyfish.jaro_winkler_similarity(data_first[key],data_second[key])
                    if key in bounded_similarity:
                        if similarity < jaro_upper_bound and similarity > jaro_lower_bound:
                            count += 1
                            changing_keys.add(key)
                    else:
                        if similarity!=1:
                            count += 1
                            changing_keys.add(key)
                    # h1 = ssdeep.hash(data_first[key])
                    # h2 = ssdeep.hash(data_second[key])
                    # fuzzy_val = ssdeep.compare(h1,h2)
                    # if fuzzy_val < fuzzy_hash_match:
                    #     count += 1
                count1 += 1
            elif key not in first_keys and key in important_keys:
                count += 1
        if count >= (0.1 * number_of_changing_keys) * number_of_keys:
            for key in data_first:
                if key in important_keys:
                    js[key] = data_first[key]
            if len(js)!=0:
                final_json[data_first["DATE_TIME"]] = js
            first = second
            second += 1
        else:
            if count1 == 0:
                first = second
                second += 1
            else:
                second += 1
    if len(final_json)>=minimum_number_of_tables:
        with open(f"{folders_path}/good_folders.txt",'a') as f:
            f.write(f"{folder}\n")
        with open(f"{folder_path}/{folder}.json",'w') as f:
            json.dump(final_json,f)
    else:
        with open(f"{folders_path}/json_compare_logging.txt",'a') as f:
            f.write(f"{folder}\n")
        bad_data += 1

print(f"folders having non usable timelines {bad_data}")

South_Vietnam
Ming_dynasty
Alupa_dynasty
Unrepresented_Nations_and_Peoples_Organization
Himan_and_Heeb
Argentina
Antarctic_Micronational_Union
Wounded_Knee_Occupation
Ilkhanate
Riograndense_Republic
Italian_Libya
Ravensburg
Slovenia
Sultanate_of_Hobyo
World_Organisation_for_Animal_Health
Gazankulu
Champa
Community_of_Latin_American_and_Caribbean_States
Canada_East
Republic_of_the_North_Solomons
East_Turkestan
Tuvalu
Darfur
Galmudug
Grand_Duchy_of_Moscow
Byzantine_Empire
Kerguelen_Islands
State_Peace_and_Development_Council
Ryukyu_Kingdom
Central_Bosnia_Canton
Federal_State_of_the_Visayas
Government_of_Free_Vietnam
Massachusetts_Bay_Colony
Aghlabids
Republic_of_South_Maluku
Niue
ASEAN
Ecuador
Peru
Bahmani_Sultanate
State_of_Buryat-Mongolia
Open_Balkan
Cape_Verde
Kingdom_of_Croatia_(Habsburg)
EU_Med_Group
Khanate_of_Kalat
Gupta_Empire
Angola
Parthian_Empire
Mongol_Empire
Sasanian_Empire
Syria
Hungary
Union_State
Yemen
Kingdom_of_Galicia
Muzaffarids_(Iran)
Celtic_nations
Lodi_dynasty
Kera

In [82]:
changing_keys

{'GDP_PPP',
 'GDP_PPP_per_capita',
 'GDP_PPP_per_capita_rank',
 'GDP_PPP_rank',
 'GDP_PPP_year',
 'GDP_nominal',
 'GDP_nominal_per_capita',
 'GDP_nominal_per_capita_rank',
 'GDP_nominal_rank',
 'GDP_nominal_year',
 'Gini',
 'Gini_rank',
 'Gini_year',
 'HDI',
 'HDI_rank',
 'HDI_year',
 'common_name',
 'largest_city',
 'leader_name1',
 'leader_name10',
 'leader_name11',
 'leader_name2',
 'leader_name3',
 'leader_name4',
 'leader_name5',
 'leader_name6',
 'leader_name7',
 'leader_name8',
 'leader_name9',
 'leader_title1',
 'leader_title10',
 'leader_title11',
 'leader_title2',
 'leader_title3',
 'leader_title4',
 'leader_title5',
 'leader_title6',
 'leader_title7',
 'leader_title8',
 'leader_title9',
 'population_census',
 'population_census_year',
 'population_density_km2',
 'population_density_rank',
 'population_density_sq_mi',
 'population_estimate',
 'population_estimate_rank',
 'population_estimate_year'}

In [45]:
less_than_two_file

['Combined_Campuses_and_Colleges_cricket_team',
 'Kalabagan_Cricket_Academy',
 'Sri_Lanka_A_cricket_team',
 'Larkana_Bulls',
 'Cricket_in_Ukraine',
 'Egypt_national_cricket_team',
 'Pakistan_International_Airlines_cricket_team',
 'Sri_Lanka_Cricket_Combined_XI',
 'New_Zealand_national_under-19_cricket_team',
 'Chicago_Catchers',
 'Chandigarh_Lions',
 'Habib_Bank_Limited_cricket_team',
 'Uva_cricket_team',
 'Galle_Guardians',
 'Risley_Cricket_Club',
 'Faisalabad_Sherdils',
 'Kurunegala_Warriors',
 'West_Hallam_White_Rose_Cricket_Club',
 'Peshawar_Haiders',
 'South_Africa_national_under-19_cricket_team',
 'North_Central_Province_cricket_team',
 'Rangers_cricket_team',
 'Sefton_Park_Cricket_Club',
 'North_East_Warriors',
 'East_and_Central_Africa_cricket_team',
 'Rhodesia_cricket_team',
 'New_Caledonia_national_cricket_team',
 'PCB_Strikers',
 'Kandy_Crusaders',
 'Kandy_Customs_Sports_Club',
 'Dorset_County_Cricket_Club',
 'Africa_XI_cricket_team',
 'Nevis_national_cricket_team',
 'Pretor

In [105]:
folders = os.listdir(folders_path)
for folder in folders:
    if folder == "json_compare_logging.txt" or folder == "good_folders.txt":
        continue
    folder_path = os.path.join(folders_path,folder)
    files = os.listdir(folder_path)
    for filename in files:
        if filename == f"{folder}.json":
            print(folder)
            os.remove(os.path.join(folder_path,filename))

Greece_national_cricket_team
Guernsey_cricket_team
Hungary_national_cricket_team
India_national_cricket_team
India_national_under-19_cricket_team
Indonesia_national_cricket_team
Ireland_cricket_team
Isle_of_Man_cricket_team
Kochi_Tuskers_Kerala
Kolkata_Knight_Riders
Lesotho_national_cricket_team
Luxembourg_national_cricket_team
Malawi_national_cricket_team
Maldives_national_cricket_team
Mali_national_cricket_team
Malta_national_cricket_team
Mexico_national_cricket_team
Mozambique_national_cricket_team
Multan_Sultans
Namibia_national_cricket_team
Nepal_national_cricket_team
Netherlands_national_cricket_team
Nigeria_national_cricket_team
Norway_national_cricket_team
Oman_national_cricket_team
Pakistan_national_cricket_team
Panama_national_cricket_team
Papua_New_Guinea_national_cricket_team
Peru_national_cricket_team
Peshawar_Zalmi
Philippines_national_cricket_team
Portugal_national_cricket_team
Pune_Warriors_India
Qatar_national_cricket_team
Rajasthan_Royals
Romania_national_cricket_team

In [96]:
less_than_two_file

['Africa_XI_cricket_team',
 'Boost_Defenders',
 'Cape_Town_Blitz_(cricket_team)',
 'Chandigarh_Lions',
 'Chennai_Superstars',
 'Chicago_Catchers',
 'Combined_Campuses_and_Colleges_cricket_team',
 'Cricket_in_Ukraine',
 'Dorset_County_Cricket_Club',
 'East_and_Central_Africa_cricket_team',
 'Egypt_national_cricket_team',
 'Faisalabad_Sherdils',
 'Galle_Guardians',
 'Habib_Bank_Limited_cricket_team',
 'Kalabagan_Cricket_Academy',
 'Kandy_Crusaders',
 'Kandy_Customs_Sports_Club',
 'Kota_Chambal_Tigers',
 'Kurunegala_Warriors',
 'Larkana_Bulls',
 'Leeward_Islands_cricket_team',
 'Merrion_Cricket_Club',
 'Nevis_national_cricket_team',
 'New_Caledonia_national_cricket_team',
 'New_Zealand_national_under-19_cricket_team',
 'North_Central_Province_cricket_team',
 'North_East_Warriors',
 'Pakistan_International_Airlines_cricket_team',
 'PCB_Strikers',
 'Peshawar_Haiders',
 'Pretoria_Capitals',
 'Rangers_cricket_team',
 'Rhodesia_cricket_team',
 'Risley_Cricket_Club',
 'Saint_Kitts_national_cric

In [50]:
final_json

{'2018-02-28T11:44:40Z': {'name': 'Pakistan',
  'coach': 'Mickey Arthur',
  'test_rank': '7th',
  'odi_rank': '6th',
  't20i_rank': '1st',
  'test_rank_best': '1st',
  'odi_rank_best': '1st',
  't20i_rank_best': '1st',
  'num_tests': '412',
  'num_tests_this_year': '0',
  'test_record': '132/122, (158 draws)',
  'test_record_this_year': '0/0 (0 draws)',
  'num_odis': '889',
  'num_odis_this_year': '5',
  'odi_record': '469/395, (8 ties, 18 no result)',
  'odi_record_this_year': '0/5 (0 ties, 0 no result)',
  'wc_apps': '11',
  'wc_best': 'Champions (1992 Cricket World Cup',
  'num_t20is': '123',
  'num_t20is_this_year': '3',
  't20i_record': '74/46, (3 ties, 0 no result)',
  't20i_record_this_year': '2/1, (0 ties, 0 no result)',
  'wt20_apps': '6',
  'wt20_best': 'Champions (2009 ICC World Twenty20'},
 '2018-08-03T17:43:34Z': {'name': 'Pakistan',
  'captain': 'Atif Aslam Ahmed',
  'coach': 'Mickey Rizvi Arthur',
  'test_rank': '1th',
  'odi_rank': '1th',
  't20i_rank': '1st',
  'test_r

In [51]:
len(final_json)

9